In [1]:
import geemap, ee
import math

# ee.Authenticate()
# ee.Initialize()
Map = geemap.Map()
# Map

In [2]:
roi01 = geemap.shp_to_ee(r'E:/work/soil_shp/points/area.shp').geometry()

In [ ]:
Map.centerObject(roi01,7);

// 加载光学、SAR数据 
// 筛选光学影像并计算 MNDWI 
var Optical_dataset = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
var SARdataset =ee.ImageCollection("COPERNICUS/S1_GRD")

var dataset1 = Optical_dataset.filterBounds(roi01)
                      .filterDate("2023-7-1","2023-9-1")
                      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',5));
var image1=dataset1.first().clip(roi01).divide(10000)
Map.addLayer(image1, {min: 0, max: 0.3,bands: ['B4', 'B3', 'B2']},"RGB")

var visParam = {  
  min: -1,   
  max: 1,  
  palette: ["FFFFFF", "CE7E45", "DF923D", "F1B555", "FCD163",   
            "99B718", "74A901", "66A000", "529400", "3E8601",   
            "207401", "056201", "004C00", "023B01", "012E01",   
            "011D01", "011301"]  
};  

var MNDWI01=image1.expression("(R-SWIR)/(R+SWIR)",{
                  "R":image1.select("B4"),"SWIR":image1.select("B11")}).rename('MNDWI')
print(MNDWI01)
 Map.addLayer(MNDWI01, visParam, "MNDWI01");
// 筛选SAR影像并计算，找到一副与光学影像日期靠近的影像 
var SARdataset = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(roi01)
        .filterDate("2023-7-1","2023-9-1")
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
        .filter(ee.Filter.eq('instrumentMode', 'IW'))
        .select('VV')
var imgVV01=ee.Image('COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20230808T103530_20230808T103555_049783_05FC90_EE91')

// 对用于水云模型的影像进行波段叠加
var sar_vv= imgVV01.clip(roi01).select('VV').rename('all_ref')             
var sar_angle=imgVV01.clip(roi01).select('angle') 
// Map.addLayer(imgVV01.select('VV'), {min: -25, max: 5})
Map.addLayer(sar_vv, {min: -25, max: 5})
// Map.addLayer(sar_angle, {min: 0, max: 90})
var all_layer = sar_vv.addBands(sar_angle).addBands(MNDWI01);

// 参照水云模型的公式对土壤后向散射系数求解
function soil_ref(image){
  // 计算植被层的双程衰减因子
  var T2 = image.expression(
    'exp(-2 * 0.3334 * MNDWI / cos(angle))', {
      'MNDWI': image.select('MNDWI'),
      'angle': image.select('angle')});
  // 植被层后向反射系数
  var veg_back = image.expression(
    '0.1449 * MNDWI * cos(angle) * (1 - T2)', {
      'MNDWI': image.select('MNDWI'),
      'angle': image.select('angle'),
      'T2': T2});
  // 计算土壤层后向反射系数
  var soil_back = image.select('all_ref').subtract(veg_back).divide(T2);
  // 返回 soil_back
  return soil_back;}
// 应用 soil_ref 函数
var soil = soil_ref(all_layer);

Map.addLayer(soil, {min: -25, max: 35},"soil")
// 导出数据 
// Export.image.toDrive({
//   image: soil,
//   description: 'soil_ref',
//   scale: 10,
//   region: roi01,
//   maxPixels: 1e13
// });